In [0]:
secrets = {
    "client_id": dbutils.secrets.get(scope="forapp-secrets", key="client-id"),
    "secret_key": dbutils.secrets.get(scope="forapp-secrets", key="secret-key"),
    "tenant_id": dbutils.secrets.get(scope="forapp-secrets", key="tenant-id")
}
print(secrets)

{'client_id': '[REDACTED]', 'secret_key': '[REDACTED]', 'tenant_id': '[REDACTED]'}


In [0]:
dbutils.secrets.list(scope="forapp-secrets")

Out[75]: [SecretMetadata(key='client-id'),
 SecretMetadata(key='secret-key'),
 SecretMetadata(key='tenant-id')]

In [0]:
# Conneting databricks to the ADLS via the app credentials created

#Step 1: creating the configs with the app credentials. A requirement to connect the app to ADLS
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": secrets["client_id"],
"fs.azure.account.oauth2.client.secret": secrets["secret_key"],
"fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{secrets['tenant_id']}/oauth2/token"}

# configs = {
#   "fs.azure.account.auth.type": "OAuth",
#   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
#   "fs.azure.account.oauth2.client.id": "<your-application-client-id>",
#   "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="<your-secret-scope>", key="<your-secret-key-for-client-secret>"),
#   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<your-tenant-id>/oauth2/token"
# }


# Step 2: Connecting the app to ADLS and specifying the data folder path
# abfss:// (Azure Blob File System Secure) ----
# source="abfss://containername@storageaccountname.dfs.core.windows.net"
# wsabs:// (Windows Storage Azure Blob Storage)
# source="wsabs://containername@storageaccountname.blob.core.windows.net"



In [0]:
# Mounts the given source directory into DBFS at the given mount point.
# dbutils.fs.mount(
# source="abfss://olypmicdata@olympicdatafalu.dfs.core.windows.net",
# mount_point="/mnt/olympicdata21",
# extra_configs=configs)

In [0]:
# List directory content using the file system 
dir = dbutils.fs.ls("/mnt/olympicdata21")
dir

Out[78]: [FileInfo(path='dbfs:/mnt/olympicdata21/raw_data/', name='raw_data/', size=0, modificationTime=1728748587000),
 FileInfo(path='dbfs:/mnt/olympicdata21/transformed_data/', name='transformed_data/', size=0, modificationTime=1728748596000)]

In [0]:
# instantiate the builtin spark object in databricks

spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [0]:
# improt packages
from pyspark.sql import functions as f 
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, StringType, DateType, DateConverter, FloatType

In [0]:
# from dfs read data
# infer.schema() to understand the data in each column and pass the appropriate datatype.
athlete = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/olympicdata21/raw_data/athletes.csv")
coaches = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/olympicdata21/raw_data/coaches.csv")
genderpersport = spark.read.format("csv").option("header", "true").load("/mnt/olympicdata21/raw_data/genderpersport.csv")
medals = spark.read.format("csv").option("header", "true").load("/mnt/olympicdata21/raw_data/medals.csv")
teams = spark.read.format("csv").option("header", "true").load("/mnt/olympicdata21/raw_data/teams.csv")

In [0]:
athlete.show(2)
athlete.printSchema()


+---------------+-------+-------------------+
|     PersonName|Country|         Discipline|
+---------------+-------+-------------------+
|AALERUD Katrine| Norway|       Cycling Road|
|    ABAD Nestor|  Spain|Artistic Gymnastics|
+---------------+-------+-------------------+
only showing top 2 rows

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
coaches.show(5)
coaches.printSchema()

+---------------+-------------+----------+-----+
|           Name|      Country|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| null|
|      ABE Junya|        Japan|Volleyball| null|
|  ABE Katsuhiko|        Japan|Basketball| null|
|   ADAMA Cherif|C�te d'Ivoire|  Football| null|
|     AGEBA Yuya|        Japan|Volleyball| null|
+---------------+-------------+----------+-----+
only showing top 5 rows

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
genderpersport.show(5)
genderpersport.printSchema()

+-------------------+------+----+-----+
|         Discipline|Female|Male|Total|
+-------------------+------+----+-----+
|     3x3 Basketball|    32|  32|   64|
|            Archery|    64|  64|  128|
|Artistic Gymnastics|    98|  98|  196|
|  Artistic Swimming|   105|   0|  105|
|          Athletics|   969|1072| 2041|
+-------------------+------+----+-----+
only showing top 5 rows

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
medals.show(5)
medals.printSchema()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
+----+--------------------+----+------+------+-----+-------------+
only showing top 5 rows

root
 |-- Rank: string (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: string (nullable = true)
 |-- Silver: string (nullable = true)
 |-- Bronze: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Rank by Total: string (nullable = true)



In [0]:
for cols in medals.columns:
    medals = medals.withColumnRenamed(cols, cols.lower()).withColumnRenamed("rank by total", "rank_by_total")

medals.printSchema()
medals

root
 |-- rank: string (nullable = true)
 |-- teamcountry: string (nullable = true)
 |-- gold: string (nullable = true)
 |-- silver: string (nullable = true)
 |-- bronze: string (nullable = true)
 |-- total: string (nullable = true)
 |-- rank_by_total: string (nullable = true)

Out[86]: DataFrame[rank: string, teamcountry: string, gold: string, silver: string, bronze: string, total: string, rank_by_total: string]

In [0]:
medals = medals.withColumn("rank", col("rank").cast(IntegerType()))\
    .withColumn("gold",col("gold").cast(IntegerType()))\
    .withColumn("silver",col("silver").cast(IntegerType()))\
    .withColumn("bronze",col("bronze").cast(IntegerType()))\
    .withColumn("total",col("total").cast(IntegerType()))\
    .withColumn("rank_by_total",col("rank_by_total").cast(IntegerType()))

In [0]:
genderpersport.show(5)
genderpersport.printSchema()

+-------------------+------+----+-----+
|         Discipline|Female|Male|Total|
+-------------------+------+----+-----+
|     3x3 Basketball|    32|  32|   64|
|            Archery|    64|  64|  128|
|Artistic Gymnastics|    98|  98|  196|
|  Artistic Swimming|   105|   0|  105|
|          Athletics|   969|1072| 2041|
+-------------------+------+----+-----+
only showing top 5 rows

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
for cols in genderpersport.columns:
    genderpersport = genderpersport.withColumnRenamed(cols, cols.lower())

genderpersport.printSchema()

root
 |-- discipline: string (nullable = true)
 |-- female: string (nullable = true)
 |-- male: string (nullable = true)
 |-- total: string (nullable = true)



In [0]:
genderpersport = genderpersport.withColumn("female", col("female").cast(IntegerType()))\
    .withColumn("male",col("male").cast(IntegerType()))\
    .withColumn("total",col("total").cast(IntegerType()))

In [0]:
teams.show(5)
teams.printSchema()

+--------+--------------+--------------------+-----+
|TeamName|    Discipline|             Country|Event|
+--------+--------------+--------------------+-----+
| Belgium|3x3 Basketball|             Belgium|  Men|
|   China|3x3 Basketball|People's Republic...|  Men|
|   China|3x3 Basketball|People's Republic...|Women|
|  France|3x3 Basketball|              France|Women|
|   Italy|3x3 Basketball|               Italy|Women|
+--------+--------------+--------------------+-----+
only showing top 5 rows

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
# find the top countries with the highest number of medals
medals.show()
medals.orderBy(f.desc("gold")).select("teamcountry","gold").show()

+----+--------------------+----+------+------+-----+-------------+
|rank|         teamcountry|gold|silver|bronze|total|rank_by_total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
 # calculate the average number of entries by gender for each discipline
avg_entries_gender = genderpersport.withColumn("avg_female", f.round(genderpersport["male"]/genderpersport["total"],2))\
                    .withColumn("avg_male", f.round(genderpersport["male"] / genderpersport["total"],2))

avg_entries_gender.show()

+--------------------+------+----+-----+----------+--------+
|          discipline|female|male|total|avg_female|avg_male|
+--------------------+------+----+-----+----------+--------+
|      3x3 Basketball|    32|  32|   64|       0.5|     0.5|
|             Archery|    64|  64|  128|       0.5|     0.5|
| Artistic Gymnastics|    98|  98|  196|       0.5|     0.5|
|   Artistic Swimming|   105|   0|  105|       0.0|     0.0|
|           Athletics|   969|1072| 2041|      0.53|    0.53|
|           Badminton|    86|  87|  173|       0.5|     0.5|
|   Baseball/Softball|    90| 144|  234|      0.62|    0.62|
|          Basketball|   144| 144|  288|       0.5|     0.5|
|    Beach Volleyball|    48|  48|   96|       0.5|     0.5|
|              Boxing|   102| 187|  289|      0.65|    0.65|
|        Canoe Slalom|    41|  41|   82|       0.5|     0.5|
|        Canoe Sprint|   123| 126|  249|      0.51|    0.51|
|Cycling BMX Frees...|    10|   9|   19|      0.47|    0.47|
|  Cycling BMX Racing|  

In [0]:
# saving the file to the mounted transformed folder 
athlete.repartition(1).write.option("header","true").csv("/mnt/olympicdata21/transformed_data/athlete")
coaches.repartition(1).write.option("header","true").csv("/mnt/olympicdata21/transformed_data/coaches")
genderpersport.repartition(1).write.option("header","true").csv("/mnt/olympicdata21/transformed_data/genderpersport")
medals.repartition(1).write.option("header","true").csv("/mnt/olympicdata21/transformed_data/medals")
teams.repartition(1).write.option("header","true").csv("/mnt/olympicdata21/transformed_data/teams")